# <center>Solving Differential Equations</center>
## <center>Using PyTorch with Runge-Kutta</center>
### <center>Dr David Race</center>

This notebook provides an introductory guide for using PyTorch to solve differential equations numerically.  This notebook uses implicit methods to solve a variety of DiffE problems using this Deep Learning toolkit.  At the conclusion of the notebook, you will see that PyTorch has these advantages:

1.   Very pythonic
2.  Easily mixes with numpy and scipy
3.   It is a powerful numeric tool on GPUs (via Google's Colaboratory)
4.  It is easy to build custom classes and functions for numerical computing
5.  Relieves the tedium of performing basic gradient computations

Overall this tool, while it is designed for Deep Learning, has a lot to offer for numerical computing.



##Background - Starting Point

While solving differential equations, we seldom know an explicit function that satisfies the differential equation; therefore, we need to use numerical methods that rely on approximations for the derivatives.  One of the popular explicit methods is Runge-Kutta; therefore, this notebook shows how to use the Runge-Kutta method for better results.

Using the implicit Runge-Kutta, we have the following:

$y^{\prime} = f(x,y)$, with $y(x_0) = y_0$.

Then given a step size $h$, we iterate through until we reach the target value for x using this algorithm:

For the target number of steps,

Set $x=x_0, y = y_0$

$k_1 = hf(x,y)$

$k_2 = hf(x + \frac{h}{2},y + \frac{k_1}{2})$

$k_3 = hf(x + \frac{h}{2},y + \frac{k_2}{2})$

$k_4 = hf(x+h, y+k_3)$

Set $x = x +h, y = y + \frac{1}{6}(k1 + 2k_2 + 2k_3 + k_4)$


##Processing

The processing will define a function to compute the y values and minimize the error.  In essence, if $F(x,y)$ is $f(x,y)$ computed in parallel, then we minimize the difference of $y^{\prime} - F(x,y)$ and leverage PyTorch's autograd to compute the best fit for a particular interval choice.  The details are shown after the environment is established.

## Set Up Environment

This section installs the basic pytorch capabilities into the Colaboratory VM.

In [0]:
from os import path
from pprint import pprint, pformat
!pip3 install -U torch
import torch
accelerator = 'cuda' if torch.cuda.is_available() else 'cpu'
#Warning:  device is used as a global name so we don't have to pass around the accelerator device
global_device = torch.device(accelerator)
print(accelerator)
pprint(global_device)

Set up the numpy environment.

In [0]:
import numpy as np
from numpy import linspace, exp
import torch

Set up the graphics environment.

In [0]:
!pip install --upgrade pandas
!pip show pandas
import pandas as pd
from pandas import DataFrame
from IPython.display import HTML, display
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
#the seaborn extension of matplotlib plots are generally "prettier"
import seaborn as sns
sns.set_style("white")

def pretty_display(x, y, x_label = 'X', y_label='Y', title='XY',num_show=25):
  max_show = min(len(x),num_show)
  x_vals = x[:max_show]
  y_vals = y[:max_show]
  xy = {x_label:x_vals, y_label:y_vals}
  df = DataFrame(xy)
  cols=[x_label, y_label]
  pData = DataFrame(df)
  pData = pData[cols]
  display(HTML('<b>'+title+'</b>'))
  display(HTML(pData.to_html()))


## Example 1 - 

$y^{\prime} = y$, 

$y(0) = 1$ ,

on the interval $[0,1]$

As you know, this is simply $y=e^x$, but this is a traditional problem to start numerical studies due to its rapid growth rate and its familiarity in mathematics and computational sciences.

###Investigations Into Implicit Runge-Kutta with PyTorch

In this particular case, we define two generic functions, compute lhs and loss

In [0]:
#Compute the left side of problem to solve
def compute_lhs(x, y, init_val):
  global global_device
  '''
  Purpose:  The function computes the approximate solution to y' = f(x,y).  Unlike
    the canonical explicit Runge-Kutta method, this computes all of the next Runge-Kutta
    values based upon the existing x and y values
  
  Required Definitions:  The function f(x,y) needs to be included in this function definition
  
  Inputs:
    x - a tensor with the current x values
    y - a tensor with the current y values (the y -values will be updated by the gradient steps)
    init_val - the initial value of y
    
  Outputs:
    The left hand side of the equation such that y' - f(x,y) = 0 and y[0] converges to the initial value

    
  TBD:  Maybe consider passing in f(x,y)
  '''
  h = x[1] - x[0]  #At some point we need to compute these for each interval so we have variable spacing
  
#   def y_prime(y):
#     assert type(y) is torch.Tensor
#     return y
  
  def f(x,y):
    assert type(x) is torch.Tensor
    assert type(y) is torch.Tensor
    return y
  
  def k1(x,y,h):
    assert type(x) is torch.Tensor
    assert type(y) is torch.Tensor
    K1 = h * f(x,y)
    return K1
  
  def k2(x,y,h, k1):
    assert type(x) is torch.Tensor
    assert type(y) is torch.Tensor
    assert type(k1) is torch.Tensor
    K2 = h * f(x + h/2.0, y + k1/2.0)
    return K2
  
  def k3(x,y,h,k2):
    assert type(x) is torch.Tensor
    assert type(y) is torch.Tensor
    assert type(k2) is torch.Tensor
    K3 = h * f(x + h/2.0, y + k2/2.0)
    return K3

  def k4(x,y,h, k3):
    assert type(x) is torch.Tensor
    assert type(y) is torch.Tensor
    assert type(k3) is torch.Tensor
    K4 = h * f(x+h, y + k3)
    return K4
  
  def F(x,y,h):
    assert type(x) is torch.Tensor
    assert type(y) is torch.Tensor
    
    K1 = k1(x,y,h)
    K2 = k2(x,y,h,K1)
    K3 = k3(x,y,h,K2)
    K4 = k4(x,y,h,K3)
    
    c = 1./6.
    Y = y + c*(K1 + 2.0 * K2 + 2.0 * K3 + K4)
    return Y
  '''
  Purpose:  Compute the left side of the equation to solve.  This implementaiton has the right side as zero.
  
  Inputs:
  x:  The x values on the interval of interest
  y:  The current approximation for the solution
  init_val:  The inital value for the left side of the interval
  h:  the uniform distance used in the solution
  
  Globals:
  global_device
  
  Output:
  c the result of the lhs computation of y' - F(x,y,h)
  '''
  
  N = y.size()[0]
  c = torch.zeros((N,1), device = global_device).double() #We generically use double precision
  c[0] = y[0] - init_val
  c[1:N] = y[1:N] - F(x,y,h)[0:(N-1)]
  return c
#
#  Compute the loss
#
def loss_xy(y, x, init_val):
  '''
  Purpose:  Compute the loss function that is used to compute the gradient for our solution to the DiffE
            NOTE:  Since the right side is computed to be zero, the lose is just the L2-norm squared
  Inputs:
  y:  The current estimate for the solution to the DiffE
  init_val:  The inital value for the left side of the interval
  h:  the uniform distance used in the solution
  
  Output:  The current loss value
  '''
  c = compute_lhs(x, y, init_val)
  e = torch.sum(c * c)
  return e
#
#  Watch for updates to these definitions as we progress through this notebook
#

In [0]:
class manage_grad_ema:
  global global_device
  '''
  Purpose:  Maintain the ema for the gradient and compute the running difference between the ema and the gradient.
  
  '''
  
  def __init__(self, ema_shape = None, num_average = 100.):
    '''
    Purpose:  initialize the gradient and the number of gradients to average
    Inputs:
    ema_shape - The shape of the gradient tensor
    num_average - The number of gradients to average using the exponential moving average
    device - the compute device for the tensor
    '''
    assert ema_shape is not None
    self.ema_shape = ema_shape
    self.device = global_device
    self.num_average = np.float(num_average)
    assert self.num_average > 0.0
    self.alpha = 2.0/(1.0 + self.num_average)
    self.ema = torch.zeros(self.ema_shape, device = self.device).double()
    self.diff_measure = 1.0
    
  def update(self, in_grad = None):
    '''
    Purpose:  Update the ema and compute the max abs of the current ema tensor and the current gradient
    
    Input
    in_grad - the current gradient
    
    Output
    The maximum of the difference between the current gradient and the current ema
    '''
    assert in_grad is not None
    assert type(in_grad) is torch.Tensor
    cur_grad_est = in_grad
    self.diff_measure = torch.max(torch.abs(self.ema - cur_grad_est))
    self.ema = cur_grad_est * self.alpha + self.ema * (1.0 - self.alpha)
    return self.diff_measure
  
  def get_diff_measure(self):
    return self.diff_measure
  def get_ema(self):
    return self.ema

In [0]:
#Simple example
start_x = 0.0
end_x = 1.0
N = 11
x = torch.tensor(linspace(start_x, end_x, N), device = global_device).double()
x = x.view((N,1))
y = torch.ones((N,1), device = global_device).double()
y[0,0]=1.0
y.requires_grad = True
pprint(x)
pprint(y)
h = (end_x - start_x)/np.float(N -1)
init_val = 1.0

e = loss_xy(y,x,init_val)
pprint(e)

optimizer1 = torch.optim.Adam([y], lr=.01)
i = 0
max_iter = 5000
epsilon = 1e-7

current_ema = manage_grad_ema(y.size())
curr_diff = 1.0

while i<=max_iter and curr_diff> epsilon:
  optimizer1.zero_grad()
  loss = loss_xy(y, x, init_val)
  loss.backward()
  optimizer1.step()
  curr_diff = current_ema.update(y.grad)
  if i % 1000 == 0:  
    print("Iteration {:d}  Loss {:.6e}  Current Diff {:.6e}".format(i,loss.data, curr_diff))
  i = i + 1

pprint(i)
pprint("Target Value of e is {:9f}".format(np.exp(1.0)))
pprint("Approx for e is {:.9f}".format(y[-1,0]))

As you can see, even with this "less than stellar" initial value for the initial guess, the result is not very bad!  Now we construct an object to use mult-grid and include a better initialization.

### Building a Simple Library

For the next step, we introduce better initial estimates and multi-grid methods as part of the study for these methods.  We will include all of the cells again just for clarity.

In [0]:
class manage_grad_ema:
  global global_device
  '''
  Purpose:  Maintain the ema for the gradient and compute the running difference between the ema and the gradient.
  
  '''
  
  def __init__(self, ema_shape = None, num_average = 100.):
    '''
    Purpose:  initialize the gradient and the number of gradients to average
    Inputs:
    ema_shape - The shape of the gradient tensor
    num_average - The number of gradients to average using the exponential moving average
    device - the compute device for the tensor
    '''
    assert ema_shape is not None
    self.ema_shape = ema_shape
    self.device = global_device
    self.num_average = np.float(num_average)
    assert self.num_average > 0.0
    self.alpha = 2.0/(1.0 + self.num_average)
    self.ema = torch.zeros(self.ema_shape, device = self.device).double()
    self.diff_measure = 1.0
    
  def update(self, in_grad = None):
    '''
    Purpose:  Update the ema and compute the max abs of the current ema tensor and the current gradient
    
    Input
    in_grad - the current gradient
    
    Output
    The maximum of the difference between the current gradient and the current ema
    '''
    assert in_grad is not None
    assert type(in_grad) is torch.Tensor
    cur_grad_est = in_grad
    self.diff_measure = torch.max(torch.abs(self.ema - cur_grad_est))
    self.ema = cur_grad_est * self.alpha + self.ema * (1.0 - self.alpha)
    return self.diff_measure
  
  def get_diff_measure(self):
    return self.diff_measure
  def get_ema(self):
    return self.ema

In [0]:
#This function is passed to the solver
#Compute the left side of problem to solve
def compute_lhs(x, y, init_val):
  global global_device
  '''
  Purpose:  The function computes the approximate solution to y' = f(x,y).  Unlike
    the canonical explicit Runge-Kutta method, this computes all of the next Runge-Kutta
    values based upon the existing x and y values
  
  Required Definitions:  The function f(x,y) needs to be included in this function definition
  
  Inputs:
    x - a tensor with the current x values
    y - a tensor with the current y values (the y -values will be updated by the gradient steps)
    init_val - the initial value of y
    
  Outputs:
    The left hand side of the equation such that y' - f(x,y) = 0 and y[0] converges to the initial value

    
  TBD:  Maybe consider passing in f(x,y)
  '''
  h = x[1] - x[0]  #At some point we need to compute these for each interval so we have variable spacing
  
#   def y_prime(y):
#     assert type(y) is torch.Tensor
#     return y
  
  def f(x,y):
    assert type(x) is torch.Tensor
    assert type(y) is torch.Tensor
    return y
  
  def k1(x,y,h):
    assert type(x) is torch.Tensor
    assert type(y) is torch.Tensor
    K1 = h * f(x,y)
    return K1
  
  def k2(x,y,h, k1):
    assert type(x) is torch.Tensor
    assert type(y) is torch.Tensor
    assert type(k1) is torch.Tensor
    K2 = h * f(x + h/2.0, y + k1/2.0)
    return K2
  
  def k3(x,y,h,k2):
    assert type(x) is torch.Tensor
    assert type(y) is torch.Tensor
    assert type(k2) is torch.Tensor
    K3 = h * f(x + h/2.0, y + k2/2.0)
    return K3

  def k4(x,y,h, k3):
    assert type(x) is torch.Tensor
    assert type(y) is torch.Tensor
    assert type(k3) is torch.Tensor
    K4 = h * f(x+h, y + k3)
    return K4
  
  def F(x,y,h):
    assert type(x) is torch.Tensor
    assert type(y) is torch.Tensor
    
    K1 = k1(x,y,h)
    K2 = k2(x,y,h,K1)
    K3 = k3(x,y,h,K2)
    K4 = k4(x,y,h,K3)
    
    c = 1./6.
    Y = y + c*(K1 + 2.0 * K2 + 2.0 * K3 + K4)
    return Y
  '''
  Purpose:  Compute the left side of the equation to solve.  This implementaiton has the right side as zero.
  
  Inputs:
  x:  The x values on the interval of interest
  y:  The current approximation for the solution
  init_val:  The inital value for the left side of the interval
  h:  the uniform distance used in the solution
  
  Globals:
  global_device
  
  Output:
  c the result of the lhs computation of y' - F(x,y,h)
  '''
  
  N = y.size()[0]
  c = torch.zeros((N,1), device = global_device).double() #We generically use double precision
  c[0] = y[0] - init_val
  c[1:N] = y[1:N] - F(x,y,h)[0:(N-1)]
  return c
#
#  Compute the loss
#
def loss_xy(y = None, x = None, init_val = None, compute_lhs = None):
  '''
  Purpose:  Compute the loss function that is used to compute the gradient for our solution to the DiffE
            NOTE:  Since the right side is computed to be zero, the lose is just the L2-norm squared
  Inputs:
  y:  The current estimate for the solution to the DiffE
  x:  The evenly spaced x values
  init_val:  The inital value for the left side of the interval
  compute_lhs:  The function to compute the left hand side of the equation so that y' - f(x,y)  = 0
  
  Output:  The current loss value
  '''
  assert y is not None
  assert type(y) is torch.Tensor
  assert y.requires_grad is True
  assert x is not None
  assert type(x) is torch.Tensor
  assert init_val is not None
  assert compute_lhs is not None
  c = compute_lhs(x, y, init_val)
  e = torch.sum(c * c)
  return e


### The Barzilai-Borwein Method

This section implements a solver that uses the Barzilai-Browein mthod.  Recall -



Without proof, we will use the <i>Barzilai-Borwein</i> method of computing a step size.  Using $\nabla F$ to denote the gradient of $F$, then we use

$x_{n+1} = x_{n} - \gamma_n * \nabla F(x_n)$, where

$\gamma_n = \frac{(x_n - x_{n-1})^T [\nabla F(x_n) - \nabla F(x_{n-1}]}{\left\Vert \nabla F(x_n) - \nabla F(x_{n-1})\right \Vert_2^2}$

This methodology is implemented in the next compute cells and augments the prvious steps for the basic solver.

In [0]:
import numpy as np
import torch

class diffe_solver_bb:
  global global_device
  '''
  The first step in more generic numerical differential equation solvers
  Purpose:  Solve a numerical differential equation defined by the inputs
  
  Inputs:
    initial_guess: an num_samplesxNx1 matrix, default is None so will be computed based upon start_x, end_x and num_intervals
    num_samples:  The number of samples to use in the model
    start_x: the starting value of x
    end_x: the ending value of x
    num_intervals: the number of inform intervals in the interval [start_x, end_x]
    lhs_function: a function to compute the left hand side of a set of finite difference set of equations, the rhs is always assumed to be zero
    init_value:  the initial value for the Differential Equation
    max_iter:  the maximum number of iterations to execute
    max_grad:  the max gradient rate.  This generally uses the Barzilai-Borwein, but this sets a max value
    epsilon:  the value that provides the ema stopping point
    device:  This is PyTorch, so we want to use the GPU whenever possible.
    
  Outputs:
    i:  the number of iterations
    y:  the approximate solution to the difference equations
  '''
  def __init__(self, initial_guess = None, start_x = 0.0, end_x = 1.0,
              lhs_function = None, cost_function = None,
              init_value = None, max_iter = 50000, epsilon=1e-6, num_exp_average = 50,
              show_progress=True):
    
    assert type(initial_guess) is torch.Tensor
    assert initial_guess is not None
    assert start_x is not None
    assert end_x is not None
    assert lhs_function is not None
    assert cost_function is not None
    assert init_value is not None
    assert max_iter is not None
    assert epsilon is not None
    assert num_exp_average is not None

    
    self.initial_guess = initial_guess
    self.device = global_device
    self.num_intervals = self.initial_guess.shape[0] - 1
    self.N = self.num_intervals + 1
    self.start_x = np.float(start_x)
    self.end_x = np.float(end_x)
    #Change to support x values in the def of y'
    self.x = torch.tensor(np.reshape(np.linspace(self.start_x, self.end_x, self.N),(self.N,1)), device = self.device).double()
    self.h = (self.end_x - self.start_x)/np.float(self.num_intervals)
    self.initial_guess = None
    if initial_guess is None:
      #This can be changes as desired to match the problem
      np.seed(0)
      self.initial_guess = torch.rand((self.N,1), device = self.device).double()
    else:
      self.initial_guess = torch.tensor(initial_guess, device = self.device).double()
    assert self.N == initial_guess.shape[0]

    self.lhs_function = lhs_function
    self.loss = cost_function
    self.init_value = np.float(init_value)
    self.max_iter = int(max_iter)
    self.epsilon = np.float(epsilon)
    self.num_exp_average = np.float(num_exp_average)
    self.alpha = 2.0/(1.0 + self.num_exp_average)

    self.show_progress = show_progress
    self.ema = manage_grad_ema(ema_shape = initial_guess.shape,num_average = self.num_exp_average)
    
  def _compute_lhs_(self, x, y):
    #need to update this when using x values
    lhs = self.lhs_function(x, y, self.init_value, self.h)
    return lhs
  
  def _stop_(self,g):
    c = self.ema.update(g)
    return c
  
  def _sample_mult_(self,a,b):
    r = torch.zeros([1], device = self.device).double()
    r = torch.mm(a,b)
    return r
  
  def _compute_step_size_(self, xn, del_xn, x_nm1, del_x_nm1):
    diff_del = del_xn - del_x_nm1
    denom = torch.sum(diff_del * diff_del)
    x_diff = xn - x_nm1
    x_diff_tp = torch.transpose(x_diff,0,1)
    num = self._sample_mult_(x_diff_tp, diff_del)
    step_size = num/denom
    #return step_size.clamp(0.0,self.max_grad)
    return step_size
    
  def solve(self):

    y = self.initial_guess
    y.requires_grad = True

    step_size = .01
#     loss = self._loss_xy_(y, self.x, self.init_value, self.h)
    loss = self.loss(y, self.x, self.init_value, self.lhs_function)
    loss.backward()
    grad_y_1 = y.grad
    
      
    y_1 = y.clone().to(self.device)
    
    with torch.no_grad():
      y = y - step_size * grad_y_1
      y.requires_grad = True
      
    i = 0
    cur_stop = 1.0
    while i<=self.max_iter and cur_stop > self.epsilon:
#       loss = self._loss_xy_(y, self.x, self.init_value, self.h)
      loss = self.loss(y, self.x, self.init_value, self.lhs_function)
      loss.backward()
      grad_y_new = y.grad.clone()

      ss = self._compute_step_size_(y, grad_y_new, y_1, grad_y_1)
      y_1 = y.clone()
      grad_y_1 = grad_y_new.clone()
 
      with torch.no_grad():
        y = y - ss * grad_y_new
        y.requires_grad = True

#       curr_loss = loss.data.numpy()
#       curr_loss_diff = np.abs(ema - curr_loss)
      cur_stop = self._stop_(y)
      if i % 500 == 0 and self.show_progress:
        pprint("*************")
        pprint(self.epsilon)
        pprint(cur_stop)
        pprint(loss)
      i = i + 1
    return i, y, cur_stop

A simple test case.

In [0]:
start_x = 0.0
end_x = 1.0
num_intervals = 50
N = num_intervals + 1
init_value = 1.0

init_guess = torch.ones((N,1), device = global_device).double()

diff_solver =  diffe_solver_bb(initial_guess = init_guess, start_x = start_x, end_x = end_x,
              lhs_function = compute_lhs, cost_function = loss_xy,
              init_value = init_value, max_iter = 5000, epsilon=1e-6, num_exp_average = 50,
              show_progress=False)

loops, y, cur_stop = diff_solver.solve()
pprint(y)
pprint(loops)
pprint(cur_stop)
pprint("Target Value of e is {:9f}".format(exp(1.0)))
pprint("Approx for e is {:.9f}".format(y[-1,0]))

This works well, so we instantiate this into a multigrid method to converge to a stronger answer

In [0]:
class diffe_solver_mm():
  global global_device
  '''
  The final step in more generic numerical differential equation solvers
  Purpose:  Solve a numerical differential equation.  This uses the diffe_solver to compute the individual number
    of intervals, but this one loops over multiple intervals with multiple samples.  The initial guess will
    just be random numbers for now!
  
  Inputs:
    start_x: the starting value of x
    end_x: the ending value of x
    num_start_intervals: the number of inform intervals in the interval [start_x, end_x]
    lhs_function: a function to compute the left hand side of a set of finite difference set of equations, the rhs is always assumed to be zero
    init_value:  the initial value for the Differential Equation
    epsilon:  the value that provides the ema stopping point
    device:  This is PyTorch, so we want to use the GPU whenever possible.
    
  Outputs:
    i:  the number of iterations
    y:  the approximate solution to the difference equations
  '''
  def __init__(self, start_x = 0.0, end_x = 1.0,
               num_start_intervals = 10, 
               lhs_function = None, cost_function = None, init_value = None,  
               epsilon=1e-6, num_exp_average = 50,
               show_progress=False):
    assert start_x is not None
    assert end_x is not None
    assert num_start_intervals is not None
    assert lhs_function is not None
    assert cost_function is not None
    assert init_value is not None
    assert epsilon is not None
    assert num_exp_average is not None
    
    self.num_intervals = int(num_start_intervals)
    self.N = self.num_intervals + 1
    self.start_x = np.float(start_x)
    self.end_x = np.float(end_x)
    self.lhs_function = lhs_function
    self.cost_function = cost_function
    self.init_value = np.float(init_value)
    self.epsilon = np.float(epsilon)
    self.num_exp_average = np.float(num_exp_average)
    self.device = global_device
    self.show_progress = show_progress
    
  def iteration_error(self, prev_est, cur_est):
    '''
    It is important to notice that cur_est has twice the number of points as the prev_est
    thus the computation only returns the error estimate based upon the common x values
    '''
    assert type(prev_est) is torch.Tensor
    assert type(cur_est) is torch.Tensor
    assert 2 * prev_est.shape[0] - 1 == cur_est.shape[0]
    N = cur_est.shape[0]
    c = prev_est - cur_est[0:N:2,:]
    return torch.sum(c*c)

  def init_guess_func(self, y):
    '''
    Purpose:  Once the first estimate is completed, double the number of intervals each time.
    Inputs
    y:  The current estimate(s)  since we have multiple samples

    Outputs
    y_vals is the estimate for doubling the number of intervals
    '''
    assert type(y) is torch.Tensor
    N = y.shape[0]
    NN = 2 * N - 1
    y_vals = torch.ones((NN,1), device = self.device).double()
    y_vals[0:NN:2,0] =y[0:N,0]
    y_vals[1:NN-1:2,0] = (y_vals[0:(NN-1):2,0] + y_vals[2:NN:2,0])/2.0
    return y_vals
    
  def solve(self):
    n_int = self.num_intervals
    N = self.N
    rhs_ = self.init_value
    
    y = torch.rand((N,1), device = self.device).double()
    y.requires_grad = True
                                           
    iter_error = 1.0
    i = 0
    
    while iter_error > self.epsilon:
      y_old = y.clone().detach()
      y = self.init_guess_func(y_old)
      diff_solve = diffe_solver_bb(initial_guess = y,  
                             start_x = self.start_x, end_x = self.end_x,
                             lhs_function = self.lhs_function, cost_function = self.cost_function, init_value= rhs_, 
                             epsilon=self.epsilon,show_progress = self.show_progress)
      n_iter, y, cur_stop = diff_solve.solve()
      iter_error = self.iteration_error(y_old, y)
      print("Number of iterations {:d} Number intervals {:d}  Current approx for right end point  {:.10f} Current stop {:.10f}".format(n_iter, y.shape[0] -  1, y[-1,0], cur_stop))
      i += 1
      
    return y

Application of this to converge to an answer that is converges with epsilon between iterations.

In [0]:
diff_solve_m = diffe_solver_mm(start_x = 0.0, end_x = 1.0, num_start_intervals = 10,
                             lhs_function = compute_lhs, cost_function = loss_xy, init_value = 1.0,
                             epsilon = 1e-10, num_exp_average = 50, show_progress = True)

y = diff_solve_m.solve()
print("Target value for right end point {:.10f}".format(np.exp(1.0)))
print("Current approx for right end point  {:.10f}".format(y[-1,0]))

This appears to be both fast and accurate, so let's move to another problem.  For different problems we only need to change the compute_lhs function to match the new f(x,y)

##Example Two - $y^{\prime} = 2y$

In this instance, we need to change the f(x,y) in the definition of compute_lhs.  Rather than pass this function into compute_lhs, the change is made in the next code cell then used in the following code cell.  Additionally, we will set the right end point to 2.0.

In [0]:
#Compute the left side of problem to solve
def compute_lhs(x, y, init_val):
  global global_device
  '''
  Purpose:  The function computes the approximate solution to y' = f(x,y).  Unlike
    the canonical explicit Runge-Kutta method, this computes all of the next Runge-Kutta
    values based upon the existing x and y values
  
  Required Definitions:  The function f(x,y) needs to be included in this function definition
  
  Inputs:
    x - a tensor with the current x values
    y - a tensor with the current y values (the y -values will be updated by the gradient steps)
    init_val - the initial value of y
    
  Outputs:
    The left hand side of the equation such that y' - f(x,y) = 0 and y[0] converges to the initial value

    
  TBD:  Maybe consider passing in f(x,y)
  '''
  h = x[1] - x[0]  #At some point we need to compute these for each interval so we have variable spacing
  
#   def y_prime(y):
#     assert type(y) is torch.Tensor
#     return y
  
  def f(x,y):
    assert type(x) is torch.Tensor
    assert type(y) is torch.Tensor
    return 2.0 * y
  
  def k1(x,y,h):
    assert type(x) is torch.Tensor
    assert type(y) is torch.Tensor
    K1 = h * f(x,y)
    return K1
  
  def k2(x,y,h, k1):
    assert type(x) is torch.Tensor
    assert type(y) is torch.Tensor
    assert type(k1) is torch.Tensor
    K2 = h * f(x + h/2.0, y + k1/2.0)
    return K2
  
  def k3(x,y,h,k2):
    assert type(x) is torch.Tensor
    assert type(y) is torch.Tensor
    assert type(k2) is torch.Tensor
    K3 = h * f(x + h/2.0, y + k2/2.0)
    return K3

  def k4(x,y,h, k3):
    assert type(x) is torch.Tensor
    assert type(y) is torch.Tensor
    assert type(k3) is torch.Tensor
    K4 = h * f(x+h, y + k3)
    return K4
  
  def F(x,y,h):
    assert type(x) is torch.Tensor
    assert type(y) is torch.Tensor
    
    K1 = k1(x,y,h)
    K2 = k2(x,y,h,K1)
    K3 = k3(x,y,h,K2)
    K4 = k4(x,y,h,K3)
    
    c = 1./6.
    Y = y + c*(K1 + 2.0 * K2 + 2.0 * K3 + K4)
    return Y
  '''
  Purpose:  Compute the left side of the equation to solve.  This implementaiton has the right side as zero.
  
  Inputs:
  x:  The x values on the interval of interest
  y:  The current approximation for the solution
  init_val:  The inital value for the left side of the interval
  h:  the uniform distance used in the solution
  
  Globals:
  global_device
  
  Output:
  c the result of the lhs computation of y' - F(x,y,h)
  '''
  
  N = y.size()[0]
  c = torch.zeros((N,1), device = global_device).double() #We generically use double precision
  c[0] = y[0] - init_val
  c[1:N] = y[1:N] - F(x,y,h)[0:(N-1)]
  return c
#

In [0]:
diff_solve_m = diffe_solver_mm(start_x = 0.0, end_x = 2.0, num_start_intervals = 10,
                             lhs_function = compute_lhs, cost_function = loss_xy, init_value = 1.0,
                             epsilon = 1e-7, num_exp_average = 50)

y = diff_solve_m.solve()
print("Target value for right end point {:.10f}".format(np.exp(4.0)))
print("Current approx for right end point  {:.10f}".format(y[-1,0]))

Overall this is a fairly solid result for the amount of effort required to achieve the final answer.  A couple of the iterationcounts were fairly large, but the answeres settled in quickly.

##Example Three - $y^{\prime} = x + 2y$

Our problem for this section is:

Solve -

$y^{\prime} = x + 2y$

$y(0) = 0.25$

on the interval $[0,2]$

It can be shown that the soltuion for this is $y = \frac{1}{2}\Big(   e^{2x} - x -\frac{1}{2} \Big)$ so the value at $y(2) ~ 26.04880$.

Once again, the changes are relati vely minor to "solve" this probelm.

In [0]:
#Compute the left side of problem to solve
def compute_lhs(x, y, init_val):
  global global_device
  '''
  Purpose:  The function computes the approximate solution to y' = f(x,y).  Unlike
    the canonical explicit Runge-Kutta method, this computes all of the next Runge-Kutta
    values based upon the existing x and y values
  
  Required Definitions:  The function f(x,y) needs to be included in this function definition
  
  Inputs:
    x - a tensor with the current x values
    y - a tensor with the current y values (the y -values will be updated by the gradient steps)
    init_val - the initial value of y
    
  Outputs:
    The left hand side of the equation such that y' - f(x,y) = 0 and y[0] converges to the initial value

    
  TBD:  Maybe consider passing in f(x,y)
  '''
  h = x[1] - x[0]  #At some point we need to compute these for each interval so we have variable spacing
  
#   def y_prime(y):
#     assert type(y) is torch.Tensor
#     return y
  
  def f(x,y):
    assert type(x) is torch.Tensor
    assert type(y) is torch.Tensor
    return x + 2.0 * y
  
  def k1(x,y,h):
    assert type(x) is torch.Tensor
    assert type(y) is torch.Tensor
    K1 = h * f(x,y)
    return K1
  
  def k2(x,y,h, k1):
    assert type(x) is torch.Tensor
    assert type(y) is torch.Tensor
    assert type(k1) is torch.Tensor
    K2 = h * f(x + h/2.0, y + k1/2.0)
    return K2
  
  def k3(x,y,h,k2):
    assert type(x) is torch.Tensor
    assert type(y) is torch.Tensor
    assert type(k2) is torch.Tensor
    K3 = h * f(x + h/2.0, y + k2/2.0)
    return K3

  def k4(x,y,h, k3):
    assert type(x) is torch.Tensor
    assert type(y) is torch.Tensor
    assert type(k3) is torch.Tensor
    K4 = h * f(x+h, y + k3)
    return K4
  
  def F(x,y,h):
    assert type(x) is torch.Tensor
    assert type(y) is torch.Tensor
    
    K1 = k1(x,y,h)
    K2 = k2(x,y,h,K1)
    K3 = k3(x,y,h,K2)
    K4 = k4(x,y,h,K3)
    
    c = 1./6.
    Y = y + c*(K1 + 2.0 * K2 + 2.0 * K3 + K4)
    return Y
  '''
  Purpose:  Compute the left side of the equation to solve.  This implementaiton has the right side as zero.
  
  Inputs:
  x:  The x values on the interval of interest
  y:  The current approximation for the solution
  init_val:  The inital value for the left side of the interval
  h:  the uniform distance used in the solution
  
  Globals:
  global_device
  
  Output:
  c the result of the lhs computation of y' - F(x,y,h)
  '''
  
  N = y.size()[0]
  c = torch.zeros((N,1), device = global_device).double() #We generically use double precision
  c[0] = y[0] - init_val
  c[1:N] = y[1:N] - F(x,y,h)[0:(N-1)]
  return c
#

In [0]:
diff_solve_m = diffe_solver_mm(start_x = 0.0, end_x = 2.0, num_start_intervals = 10,
                             lhs_function = compute_lhs, cost_function = loss_xy, init_value = .25,
                             epsilon = 1e-7, num_exp_average = 50)

y = diff_solve_m.solve()
print("Target value for right end point {:.10f}".format(.5*(np.exp(4.0)-2.0 - .5)))
print("Current approx for right end point  {:.10f}".format(y[-1,0]))

##  Conclusion

This demonstrates that autograd and PyTorch have a large potential for doing investigations solving differential equations.  Using the natural expressions for the estimations makes the code easy to read and implement.  This notebook put little to no effort in optimizing the solution, but the convergence was good for all of the problems.  This approach is a practical method for early investigations that can then be converted to production applications.